In [1]:
import numpy as np
import pandas as pd
import json
import pickle
import category_encoders as ce

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.style.use('default')

In [3]:
import sys, os
sys.path.append(os.path.join(os.path.abspath(''), '..', 'shared_libs'))
import data_transform

In [4]:
df = pd.read_csv('data/data_target_cleared.csv')
df.head()

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,for sale,NaN,multi-family,803 Passmore St,2 Baths,"{'atAGlanceFacts': [{'factValue': '1950', 'fac...",yes,Philadelphia,"[{'rating': ['3/10', '2/10', '3/10'], 'data': ...","1,350 sqft",19111,3 Beds,PA,2.0,NaN,NaN,PAPH853202,195000.0
1,for sale,NaN,lot/land,3609 Summit Ave,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Greensboro,"[{'rating': ['5/10', '5/10', '4/10'], 'data': ...",NaN,27405,NaN,NC,NaN,NaN,NaN,930640,199000.0
2,Active,NaN,Land,4011 Valley Vista Dr,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Lowell,"[{'rating': ['7', '6'], 'data': {'Distance': [...",0,49331,NaN,MI,NaN,NaN,NaN,19058608,189900.0
3,For sale,NaN,Single Family,118 S 17th St,Bathrooms: 2,"{'atAGlanceFacts': [{'factValue': '1909', 'fac...",NaN,Allentown,"[{'rating': ['2/10', '2/10', '2/10'], 'data': ...","Total interior livable area: 1,617 sqft",18104,3 bd,PA,2.0,628531,NaN,NaN,189000.0
4,for sale,NaN,single-family home,6820 Quincy St,4 Baths,"{'atAGlanceFacts': [{'factValue': '1925', 'fac...",yes,Philadelphia,"[{'rating': ['3/10', '1/10'], 'data': {'Distan...","5,013 sqft",19119,6 Beds,PA,3.0,NaN,NaN,PAPH851112,579000.0


In [5]:
df = data_transform.clear_data_base_line(
    df, 
    '../shared_libs/data/default_values.pkl',
    can_drop_rows=True, 
    force_rebuild_cached_data=True
)

Считываем статистические данные о населении в городах США ([источник](https://www.kaggle.com/datasets/sergejnuss/united-states-cities-database))

In [6]:
df_us_pop = pd.read_csv('data/uscities.csv')
df_us_pop.head(5)

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36081,Queens,40.6943,-73.9249,18972871,10768.2,shape,False,True,America/New_York,1,11229 11226 11225 11224 11223 11221 11220 1138...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1141,-118.4068,12121244,3267.6,shape,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90035 90034 9003...,1840020491
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8375,-87.6866,8595181,4576.6,shape,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7840,-80.2101,5711945,4945.7,shape,False,True,America/New_York,1,33128 33129 33125 33126 33127 33149 33144 3314...,1840015149
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7935,-96.7667,5668165,1522.2,shape,False,True,America/Chicago,1,75287 75098 75234 75254 75251 75252 75253 7503...,1840019440


In [7]:
df_us_pop['city'] = df_us_pop['city'].apply(lambda x: str.lower(x))

In [8]:
df_us_pop = df_us_pop[['state_id', 'city', 'population', 'density']]
df_us_pop.drop_duplicates(inplace=True)

In [9]:
df_us_pop = df_us_pop.groupby(['state_id', 'city']).agg({'population': 'median', 'density': 'median'}).reset_index()

In [10]:
df_pop = pd.merge(
    df,
    df_us_pop,
    left_on=['state', 'city'],
    right_on=['state_id', 'city'],
    how='left'
).drop(['state_id'], axis=1)

In [11]:
df = df_pop.copy()

Объединяем данные из датасета о статистике городов США с нашим дата сетом (ключ пересечения *state, city*)

In [12]:
not_found_cities = (df[df['population'].isna()][['state', 'city']].value_counts()/df.shape[0]*100).index

In [13]:
not_found_cities

MultiIndex([('FL',    'saint petersburg'),
            ('NY',            'flushing'),
            ('FL',       'st petersburg'),
            ('NY',             'jamaica'),
            ('NC',       'winston salem'),
            ('FL',    'port saint lucie'),
            ('FL',         'saint johns'),
            ('MO',         'saint louis'),
            ('TN',             'antioch'),
            ('NJ',            'lakewood'),
            ...
            ('PA',          'mt lebanon'),
            ('PA',         'new hanover'),
            ('MA',      'indian orchard'),
            ('MA',       'cherry valley'),
            ('TN',           'arrington'),
            ('TN',            'bellevue'),
            ('TN',       'college grove'),
            ('MA', 'boston (dorchester)'),
            ('TN',            'corryton'),
            ('AL',              'bryant')],
           names=['state', 'city'], length=621)

Для городов не найденных во внешних данных, заполняем пропуски медианными значениями.

In [ ]:
state_medians = df[~df['population'].isna()].groupby('state').agg({'population': 'median', 'density': 'median'})

In [ ]:
mask = df['population'].isna()
df.loc[mask, 'population'] = df[mask]['state'].apply(lambda x: np.NaN if x not in state_medians.index else state_medians.loc[x, 'population'])

In [ ]:
mask = df['density'].isna()
df.loc[mask, 'density'] = df[mask]['state'].apply(lambda x: np.NaN if x not in state_medians.index else state_medians.loc[x, 'density'])

In [ ]:
df = df[~df['population'].isna()].copy()

Преобразуем числовые поля *population* и *density* в категориальные (линейным разбиением диапазона на равные отрезки и выбором номера отрезка, в который попадпет значение).

In [ ]:
min_val = df['population'].min()
max_val = df['population'].max()
n_cats = 15
df['population_cat'] = 0
df['population_cat'] = df['population'].apply(lambda x: data_transform.convert_to_ord_cat(x, min_val, max_val, n_cats))

In [ ]:
min_val = df['density'].min()
max_val = df['density'].max()
n_cats = 15
df['density_cat'] = 0
df['density_cat'] = df['density'].apply(lambda x: data_transform.convert_to_ord_cat(x, min_val, max_val, n_cats))

In [ ]:
df = df.drop(['population', 'density'], axis=1)

In [ ]:
bin_encoder = ce.BinaryEncoder(cols=['population_cat'])
type_bin = bin_encoder.fit_transform(df['population_cat'])
df = pd.concat([df, type_bin], axis=1).drop('population_cat', axis=1)

bin_encoder = ce.BinaryEncoder(cols=['density_cat'])
type_bin = bin_encoder.fit_transform(df['density_cat'])
df = pd.concat([df, type_bin], axis=1).drop('density_cat', axis=1)

Полученные категориальные признаки (*population_cat* и *density_cat*) кодируем, используя BinaryEncoder.